# notebook-ai Demo

This notebook demonstrates the `%%prompt` magic for LLM-assisted workflows.

**Key concepts:**
- Use `{variable}` to give the LLM access to data
- Use `@tool` to define functions the LLM can call
- Tool parameters must be simple types (str, int, float, bool)

In [ ]:
# Load the extension
%load_ext notebook_ai
from notebook_ai import tool
import random
import requests

## Simple Tool Examples

These tools use only simple types (str, int, float, bool) which Claudette supports natively.

In [ ]:
# Simple tools with basic types only

@tool
def calculate(expression: str) -> str:
    """
    Evaluate a mathematical expression safely.
    
    Args:
        expression: A math expression like '2 + 2' or '15 * 7'
    """
    # Only allow safe math operations
    allowed = set('0123456789+-*/.() ')
    if not all(c in allowed for c in expression):
        return "Error: Invalid characters in expression"
    try:
        result = eval(expression)
        return f"{expression} = {result}"
    except Exception as e:
        return f"Error: {e}"

@tool  
def get_weather(city: str) -> str:
    """
    Get current weather for a city using the Open Meteo API.
    
    Args:
        city: Name of the city to get weather for
    """
    try:
        # First, geocode the city name to coordinates
        geo_url = f"https://geocoding-api.open-meteo.com/v1/search?name={city}&count=1"
        geo_response = requests.get(geo_url)
        geo_data = geo_response.json()
        
        if "results" not in geo_data or len(geo_data["results"]) == 0:
            return f"Error: Could not find location '{city}'"
        
        location = geo_data["results"][0]
        lat, lon = location["latitude"], location["longitude"]
        country = location.get("country", "")
        
        # Now get the weather
        weather_url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m,weather_code"
        weather_response = requests.get(weather_url)
        weather_data = weather_response.json()
        
        temp = weather_data["current"]["temperature_2m"]
        weather_code = weather_data["current"]["weather_code"]
        
        # Map weather codes to descriptions
        weather_descriptions = {
            0: "clear sky", 1: "mainly clear", 2: "partly cloudy", 3: "overcast",
            45: "foggy", 48: "depositing rime fog",
            51: "light drizzle", 53: "moderate drizzle", 55: "dense drizzle",
            61: "slight rain", 63: "moderate rain", 65: "heavy rain",
            71: "slight snow", 73: "moderate snow", 75: "heavy snow",
            80: "slight rain showers", 81: "moderate rain showers", 82: "violent rain showers",
            95: "thunderstorm", 96: "thunderstorm with slight hail", 99: "thunderstorm with heavy hail"
        }
        condition = weather_descriptions.get(weather_code, "unknown")
        
        return f"Weather in {city}, {country}: {temp}°C, {condition}"
    except Exception as e:
        return f"Error fetching weather: {e}"

@tool
def roll_dice(count: int = 5) -> str:
    """
    Roll six-sided dice for Yahtzee and return the results.
    
    Args:
        count: Number of dice to roll (default 5 for Yahtzee)
    """
    rolls = [random.randint(1, 6) for _ in range(count)]
    total = sum(rolls)
    if count == 1:
        return f"Rolled: {rolls[0]}"
    return f"Rolled {count} dice: {rolls} (total: {total})"

In [ ]:
%%prompt
What's 127 * 43? Use {calculate} to compute this.

In [ ]:
%%prompt
Check the weather in London and Edinburgh using {get_weather}, then tell me which city is warmer.

In [ ]:
%%prompt
I'm playing Yahtzee! Use {roll_dice} to roll my 5 dice, then tell me what scoring options I have.

## Working with DataFrames

For complex types like DataFrames, use `str` as the type hint. The variable name gets resolved to the actual object automatically.

In [ ]:
# Create some sample data
import pandas as pd

sales = pd.DataFrame({
    'product': ['Widget A', 'Widget B', 'Widget C', 'Widget D'],
    'units': [150, 89, 203, 67],
    'price': [19.99, 34.50, 9.99, 74.99],  # Prices in GBP
    'region': ['North', 'South', 'North', 'East']
})

sales['revenue'] = sales['units'] * sales['price']
sales

In [ ]:
# Define a tool - parameters must be simple types (str, int, float, bool)
# Use str for variable names - they get resolved to actual objects automatically
@tool
def top_products(df: str, n: int = 3) -> str:
    """
    Return the top N products by revenue from a dataframe.
    
    Args:
        df: Name of the dataframe variable containing product data
        n: Number of top products to return
    """
    top = df.nlargest(n, 'revenue')[['product', 'revenue']]
    return top.to_string()

In [ ]:
%%prompt
What's the total revenue in {sales}? Which region is performing best?

In [ ]:
%%prompt
Use {top_products} to find the best performers in {sales}, then explain why they might be successful.

## Code Improvement Mode

Use `--code` to get just the improved code without explanation - perfect for copy-paste workflows.

In [ ]:
# Some code that could be improved
def proc(d):
    r = []
    for i in d:
        if i > 0:
            r.append(i * 2)
    return r

In [ ]:
%%prompt --code
Improve the `proc` function above: use better names, add type hints, and make it more Pythonic.

## Caching

Responses are cached automatically. Re-running cells returns cached results instantly (marked with "cached response"). Use `--no-cache` to force a fresh API call, or `clear_cache()` to reset.

In [ ]:
# Check cache stats
from notebook_ai import cache_stats, clear_cache
cache_stats()

In [ ]:
%%prompt --no-cache
Roll 5 dice using {roll_dice} - this will always make a fresh API call.